# 2024 COMP90042 Project
*Yaoyuan Zhang ID:1410123*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
!pip install rank_bm25

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [7]:
import numpy as np
import pandas as pd

#libraries for getting data and extracting
import os
import urllib.request
import tarfile
import json
import pandas as pd
import numpy as np
from tqdm import tqdm


#libraries for text preprocessing
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

#libraries for keyword extraction with tf-idf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.sparse import coo_matrix

#libraries for reading and writing files
import pickle

#libraries for BM25
from rank_bm25 import BM25Okapi

# Load Data
train_claims = pd.read_json('/content/Fact_Checking_System/data/train-claims.json', lines=True)
dev_claims = pd.read_json('/content/Fact_Checking_System/data/dev-claims.json', lines=True)
dev_claims_baselines = pd.read_json('/content/Fact_Checking_System/data/dev-claims-baseline.json', lines = True)
evidence =  pd.read_json('/content/Fact_Checking_System/data/evidence.json', lines = True)
test_claims = pd.read_json('/content/Fact_Checking_System/data/test-claims-unlabelled.json', lines=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


ValueError: Unmatched ''"' when when decoding 'string'

In [ ]:
train_claims

,claim-1937,claim-126,claim-2510,claim-2021,claim-2449,claim-851,claim-2773,claim-949,claim-1019,claim-2834,...,claim-2076,claim-2508,claim-2049,claim-1333,claim-1316,claim-1504,claim-243,claim-2302,claim-502,claim-3093
0,{'claim_text': 'Not only is there no scientifi...,{'claim_text': 'El Niño drove record highs in ...,"{'claim_text': 'In 1946, PDO switched to a coo...",{'claim_text': 'Weather Channel co-founder Joh...,"{'claim_text': '""January 2008 capped a 12 mont...",{'claim_text': 'The last time the planet was e...,{'claim_text': 'Tree-ring proxy reconstruction...,{'claim_text': 'Under the most ambitious scena...,{'claim_text': 'An additional kick was supplie...,{'claim_text': 'When stomata-derived CO2 (red)...,...,{'claim_text': 'Snowfall is increasing in the ...,"{'claim_text': '""The Pacific Decadal Oscillati...",{'claim_text': 'The Millennium Drought startin...,{'claim_text': '“The oceans will never become ...,{'claim_text': 'In pushing too hard for the ca...,{'claim_text': 'Climate scientists say that as...,{'claim_text': 'In its 5th assessment report i...,"{'claim_text': 'Since the mid 1970s, global te...",{'claim_text': 'But abnormal temperature spike...,{'claim_text': 'Sending oscillating microwaves...


In [ ]:
claim_texts = train_claims.apply(lambda col: col.map(lambda x: x.get('claim_text') if isinstance(x, dict) else None))

In [ ]:
claim_texts

,claim-1937,claim-126,claim-2510,claim-2021,claim-2449,claim-851,claim-2773,claim-949,claim-1019,claim-2834,...,claim-2076,claim-2508,claim-2049,claim-1333,claim-1316,claim-1504,claim-243,claim-2302,claim-502,claim-3093
0,Not only is there no scientific evidence that ...,El Niño drove record highs in global temperatu...,"In 1946, PDO switched to a cool phase.",Weather Channel co-founder John Coleman provid...,"""January 2008 capped a 12 month period of glob...",The last time the planet was even four degrees...,Tree-ring proxy reconstructions are reliable b...,"Under the most ambitious scenarios, they found...",An additional kick was supplied by an El Niño ...,When stomata-derived CO2 (red) is compared to ...,...,Snowfall is increasing in the fall and winter ...,"""The Pacific Decadal Oscillation (PDO) is a te...",The Millennium Drought starting in 1997 and en...,“The oceans will never become acid because the...,In pushing too hard for the case that global w...,Climate scientists say that aspects of the cas...,"In its 5th assessment report in 2013, the IPCC...","Since the mid 1970s, global temperatures have ...",But abnormal temperature spikes in February an...,Sending oscillating microwaves from an antenna...


In [ ]:
evidence

,evidence-0,evidence-1,evidence-2,evidence-3,evidence-4,evidence-5,evidence-6,evidence-7,evidence-8,evidence-9,...,evidence-1208817,evidence-1208818,evidence-1208819,evidence-1208820,evidence-1208821,evidence-1208822,evidence-1208823,evidence-1208824,evidence-1208825,evidence-1208826
0,"John Bennet Lawes, English entrepreneur and ag...",Lindberg began his professional career at the ...,``Boston (Ladies of Cambridge)'' by Vampire We...,"Gerald Francis Goyer (born October 20, 1936) w...",He detected abnormalities of oxytocinergic fun...,With peak winds of 110 mph (175 km/h) and a mi...,He is currently a professor of piano at the Un...,"In addition to known and tangible risks, unfor...",Sir John Sherbrooke was able to hold her off f...,Aslan Tlebzu (Аслъан ЛIыбзэу [- adyaːsɬaːn ɬʼə...,...,"They reached the final, where a record attenda...","Caria ([ˈkɛəriə] from Greek : Καρία, Karia, Ka...",Contemporary assumptions made about the circum...,"Richard Arnold Kraus (August 27, 1937) is a Ma...",When his second wife died in 1874 Harry Grey e...,Also on the property is a contributing garage ...,| class = ``fn org'' | Fyrde | | | | 6110 | | ...,"Dragon Storm (game), a role-playing game and c...",It states that the Zeriuani ``which is so grea...,The storyline revolves around a giant plesiosa...


In [ ]:
def preprocess(text):
    # 假定 text 是字符串类型
    if not isinstance(text, str):
        return text  # 如果不是，直接返回原值
    # 定义停用词
    stop_words = set(stopwords.words("english"))
    # 其余的处理步骤
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    text = re.sub("(\\d|\\W)+"," ",text)
    text = text.split()
    ps = PorterStemmer()
    text = [ps.stem(word) for word in text if not word in stop_words]
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(word) for word in text if not word in stop_words]
    text = " ".join(text)
    return text

In [ ]:
batch_size = 10000
preprocessed_batches = []

for start in range(0, len(evidence), batch_size):
    end = min(start + batch_size, len(evidence))
    batch = evidence.iloc[start:end]
    preprocessed_batch = batch.apply(preprocess)
    preprocessed_batches.append(preprocessed_batch)

df_preprocessed = pd.concat(preprocessed_batches, ignore_index=True)

In [ ]:
df_preprocessed

,evidence-0,evidence-1,evidence-2,evidence-3,evidence-4,evidence-5,evidence-6,evidence-7,evidence-8,evidence-9,...,evidence-1208817,evidence-1208818,evidence-1208819,evidence-1208820,evidence-1208821,evidence-1208822,evidence-1208823,evidence-1208824,evidence-1208825,evidence-1208826
0,"John Bennet Lawes, English entrepreneur and ag...",Lindberg began his professional career at the ...,``Boston (Ladies of Cambridge)'' by Vampire We...,"Gerald Francis Goyer (born October 20, 1936) w...",He detected abnormalities of oxytocinergic fun...,With peak winds of 110 mph (175 km/h) and a mi...,He is currently a professor of piano at the Un...,"In addition to known and tangible risks, unfor...",Sir John Sherbrooke was able to hold her off f...,Aslan Tlebzu (Аслъан ЛIыбзэу [- adyaːsɬaːn ɬʼə...,...,"They reached the final, where a record attenda...","Caria ([ˈkɛəriə] from Greek : Καρία, Karia, Ka...",Contemporary assumptions made about the circum...,"Richard Arnold Kraus (August 27, 1937) is a Ma...",When his second wife died in 1874 Harry Grey e...,Also on the property is a contributing garage ...,| class = ``fn org'' | Fyrde | | | | 6110 | | ...,"Dragon Storm (game), a role-playing game and c...",It states that the Zeriuani ``which is so grea...,The storyline revolves around a giant plesiosa...


## ORGANIZE

### Sort Coordinate Matrix

#### Input
- TF-IDF coordinate representation

#### Output
- TF-IDF items sorted in descending order — with items with the highest scores at the top.


In [ ]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)


In [ ]:
# get top n words with highest tf-idf scores
# input:
# 1. feature_names = vocabulary
# 2. sorted_items = tf-idf vectors sorted in descending order
# 3. topN = # of keywords you would like extract from text
# output: dictionary of topN # words with highest tf-idf scores in the text (key) and their corresponding tf-idf scores (value)
# Gets keyword names and their tf-idf scores of topN items
def extract_topn_from_vector(feature_names, sorted_items, topN):
    #use only topn items from vector
    sorted_items = sorted_items[:topN]

    score_vals = []
    feature_vals = []
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    return results

# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*